# Геолокация сообщений

#### Импорт библиотек

In [1]:
import warnings
warnings.filterwarnings(action="ignore",category=DeprecationWarning, append=True)
warnings.simplefilter(action="ignore", category=DeprecationWarning, append=True)

In [2]:
import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka.src.geocoder.geocoder import Geocoder

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


#### Загрузка данных

In [3]:
import pandas as pd

df = pd.read_csv('/Users/test/Documents/code/sloyka/sloyka/sample_data/sample_data.csv', index_col=0)

In [4]:
df.head(3)

,message_id,Дата и время,Текст комментария
0,0,2023.01.26 16:32,Здравствуйте! В Санкт-Петербурге нет Генеральн...
1,1,2023.01.26 11:55,[club143265175|Центральный район Санкт-Петербу...
2,2,2023.01.28 12:39,"1) Фурштатская, 19 Отслоение штукатурного слоя..."


In [ ]:
osm_id = 337422 # Saint Petersburg
geocoder = Geocoder(df, osm_id=osm_id, city_tags={'place':['state']}, text_column_name='Текст комментария')


In [ ]:
result = geocoder.run(group_column=None)

In [7]:
result.sample(3)

/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


,Текст комментария,Street,Numbers,Toponyms,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,message_id,Дата и время
11,Это просто кошмар какой-то. Дворы вообще не уб...,марата,,None,улица Марата Санкт-Петербург Россия,[улица Марата Санкт-Петербург Россия],улица Марата Санкт-Петербург Россия,улица Марата,"улица Марата, Владимирский округ, Санкт-Петерб...",POINT (30.34462 59.92130),11,2023.01.15 15:37
64,[club148997934|ГАТИ | Правительство Санкт-Пете...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,64,2022.12.19 17:01
194,"А почему нет акции на Жуковского, Восстания, М...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,186,2022.11.10 15:30


In [8]:
from IPython.core.display import HTML

HTML(result.explore(tiles='cartodbdark_matter', marker_kwds={'radius':10})._repr_html_())

/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager 